In [1]:
import time
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException  # Import the exception
import pandas as pd

In [3]:

# Automatically download and install Chromedriver
chromedriver_autoinstaller.install()

# Create a new instance of the Chrome driver
driver = webdriver.Chrome()

# URL of the page you want to visit
url = "https://mer.markit.com/br-reg/public/index.jsp?entity=retirement&srd=false&sort=account_name&dir=ASC&start=0&entity_domain=Markit&additionalCertificationId=&acronym=PV&standardId=100000000000004&categoryId=100000000000001&unitClass="
# Open the URL in the Chrome browser
driver.get(url)

# Explicitly wait for the table header to be present
header_xpath = "/html/body/div/div[2]/div[2]/table/tbody/tr[1]/th"
header_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, header_xpath)))

# Extract headers text
headers = [header.text for header in header_elements]

# Add new columns for projectID and country
headers.extend(['projectID', 'country'])

# Initialize an empty list to store row data
data = []
i=0
while True:
    # Find all rows in the table
    row_xpath = "/html/body/div/div[2]/div[2]/table/tbody/tr[position()>1]"

    # Handle StaleElementReferenceException by re-finding the row elements
    try:
        row_elements = driver.find_elements(By.XPATH, row_xpath)
    except StaleElementReferenceException:
        print("Stale element reference. Refetching row elements.")
        continue

    # Loop through each row
    for row_element in row_elements:
        try:
            # Find all columns in the current row
            col_elements = row_element.find_elements(By.XPATH, "td")
            # Extract text from each column and add to the row data list
            row_data = [col.text for col in col_elements]

            try:
                # Click the link in the last column
                link = row_element.find_element(By.XPATH, "td[last()]/a")
                link.click()

                # Switch to the new tab
                driver.switch_to.window(driver.window_handles[1])

                # Wait for the required element on the new page
                project_id_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div[2]/div[1]/div/h3/span"))
                )
                project_id = project_id_element.text

                # Extract country information
                country_element = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[3]/div/table/tbody/tr[3]/td")
                country = country_element.text

                # Add projectID and country to the row data
                row_data.extend([project_id, country])

                # Close the new tab
                driver.close()

                # Switch back to the original tab
                driver.switch_to.window(driver.window_handles[0])

            except Exception as e:
                # Handle errors (e.g., if the link is not present)
                print(f"Error processing row: {e}")
                continue

            # Add row data to the overall data list
            data.append(row_data)

        except StaleElementReferenceException:
            # Handle StaleElementReferenceException by re-finding the row elements
            print("Stale element reference. Refetching row elements.")
            break

    # Check if the button/link for the next page exists
    next_page_button = driver.find_elements(By.XPATH, "/html/body/div/div[2]/div[2]/ul[2]/li[3]/a")

    try:
        if not next_page_button:
            # No more pages, break the loop
            break

        # Click the button/link for the next page
        next_page_button[0].click()
        time.sleep(10)
        i +=1
        print(i)
    except Exception as e:
        # Handle the error when clicking the next page button
        print(f"Error clicking next page button: {e}")
        break  # Break out of the loop if an error occurs while clicking the next page button

# Create a Pandas DataFrame using the headers and row data
df = pd.DataFrame(data, columns=headers)

# Print the DataFrame
print(df)

# Close the Chrome driver
driver.quit()


Error processing row: Message: no such element: Unable to locate element: {"method":"xpath","selector":"td[last()]/a"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D46EE3+174339]
	(No symbol) [0x00C70A51]
	(No symbol) [0x00986FF6]
	(No symbol) [0x009B9876]
	(No symbol) [0x009B9C2C]
	(No symbol) [0x009B2631]
	(No symbol) [0x009D7054]
	(No symbol) [0x009B25B0]
	(No symbol) [0x009D7414]
	(No symbol) [0x009EA104]
	(No symbol) [0x009D6DA6]
	(No symbol) [0x009B1034]
	(No symbol) [0x009B1F8D]
	GetHandleVerifier [0x00DE4B1C+820540]
	sqlite3_dbdata_init [0x00EA53EE+653550]
	sqlite3_dbdata_init [0x00EA4E09+652041]
	sqlite3_dbdata_init [0x00E997CC+605388]
	sqlite3_dbdata_init [0x00EA5D9B+656027]
	(No symbol) [0x00C7FE6C]
	(No symbol) [0x00C783B8]
	(No symbol) [0x00C784DD]
	(No symbol) [0x00C65818]
	BaseThreadInitThunk [0x7

Error processing row: Message: element click intercepted: Element <a target="_BLANK" href="project.jsp?project_id=104000000014148">...</a> is not clickable at point (1005, 7). Other element would receive the click: <td>...</td>
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00D46EE3+174339]
	(No symbol) [0x00C70A51]
	(No symbol) [0x00986FF6]
	(No symbol) [0x009BE48E]
	(No symbol) [0x009BD09E]
	(No symbol) [0x009BB5F8]
	(No symbol) [0x009BAD7F]
	(No symbol) [0x009B2B4E]
	(No symbol) [0x009D700C]
	(No symbol) [0x009B25B0]
	(No symbol) [0x009D7414]
	(No symbol) [0x009EA104]
	(No symbol) [0x009D6DA6]
	(No symbol) [0x009B1034]
	(No symbol) [0x009B1F8D]
	GetHandleVerifier [0x00DE4B1C+820540]
	sqlite3_dbdata_init [0x00EA53EE+653550]
	sqlite3_dbdata_init [0x00EA4E09+652041]
	sqlite3_dbdata_init [0x00E997CC+605388]
	sqlite3_dbdata_init [0x00EA5D9B+656027]
	(No symbol) [0x00C7FE6C]
	(No symbol) [0x00C783B8]
	(No symbol) [0x00C784DD]
	(No symbol) [0x00C65818]
	BaseThrea

1
Error processing row: Message: no such element: Unable to locate element: {"method":"xpath","selector":"td[last()]/a"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D46EE3+174339]
	(No symbol) [0x00C70A51]
	(No symbol) [0x00986FF6]
	(No symbol) [0x009B9876]
	(No symbol) [0x009B9C2C]
	(No symbol) [0x009B2631]
	(No symbol) [0x009D7054]
	(No symbol) [0x009B25B0]
	(No symbol) [0x009D7414]
	(No symbol) [0x009EA104]
	(No symbol) [0x009D6DA6]
	(No symbol) [0x009B1034]
	(No symbol) [0x009B1F8D]
	GetHandleVerifier [0x00DE4B1C+820540]
	sqlite3_dbdata_init [0x00EA53EE+653550]
	sqlite3_dbdata_init [0x00EA4E09+652041]
	sqlite3_dbdata_init [0x00E997CC+605388]
	sqlite3_dbdata_init [0x00EA5D9B+656027]
	(No symbol) [0x00C7FE6C]
	(No symbol) [0x00C783B8]
	(No symbol) [0x00C784DD]
	(No symbol) [0x00C65818]
	BaseThreadInitThunk [0

In [ ]:
df = pd.DataFrame(data, columns=headers)
df.to_csv(r'D:\plan vivo retirements_till page 21-.csv')